<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2026/blob/main/TubularReactorBandBroadening2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
%run che5.ipynb

In [ ]:
N = 100
q=100. #L/min
Cain = 1. #mol/L
Tin = 350. #K
totV = 100. #L
V = totV/N
rho = 1e3 #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min
UAtot = 5e4 #J/(min K)
UA = UAtot/N

Tc0 = 300. #K
Ca0 = 0.5 #mol/L
T0 = 350. #K

cin0 = 1. #mol/L

c0 = np.full(N,0.5)
T0 = np.full(N, 350.)
tend=10

In [ ]:
def sigmoid(x, x0=0, width=1.):
    return sp.special.expit((x-x0)/width)

def pulse(x, x1, x2, c1, c2, width=1.):
    return c1 + (c2-c1)*sigmoid(x, x0=x1, width=width) - (c2-c1)*sigmoid(x, x0=x2, width=width)

In [ ]:
x = np.linspace(0,10,1000)
fig = make_subplots()
fig.add_scatter(x=x, y=pulse(x, 1., 2., 1., 4., 0.01))

In [ ]:
def rhs(t, vec):
    cin = pulse(t, 0., 0.2, 1., 4., 0.01)
    c = vec
    dc = np.zeros(N)
    dc[0] = q*(cin - c[0])/V
    dc[1:] = q*(c[:-1] - c[1:])/V
    return dc

In [ ]:
N_times = 100
dt = tend/N_times
tplot = np.linspace(0,tend,N_times)
res=sp.integrate.solve_ivp(rhs, (0,tend), c0, method='Radau', dense_output=True)
c_profiles = res.sol(tplot)

In [ ]:
fig = make_subplots()
for t_index in range(N_times):
    fig.add_scatter(x = np.arange(N), y = c_profiles[:, t_index], name=f't={dt*t_index:.2f}')
fig